In [ ]:
# downloading dataset
import pandas as pd
df = pd.read_csv('Covid19_ts.csv',parse_dates=['Date'])
df['Date'] = pd.to_datetime(df['Date'])
df = pd.DataFrame(df, columns=['Date','Confirmed']).set_index('Date')
train = df.iloc[:200, :]
train.index = pd.to_datetime(train.index)# convert index to datetime format
df

In [ ]:
#simple exponential smoothing 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
import numpy as np
train_data = pd.Series(train.Confirmed, train.index)

# building ES model 
model = SimpleExpSmoothing(train_data, initialization_method="heuristic")


# The smoothing_level value of the simple exponential smoothing, if the value is set then this value will be used as the value.
fm1 = model.fit(smoothing_level=.2)#fitting the model
fcast_1 = fm1.forecast(9).rename(r"$\alpha=0.2$")# The number of out of sample forecasts from the end of the sample.
fm2 = model.fit(smoothing_level=.9)
fcast_2 = fm2.forecast(9).rename(r"$\alpha=0.9$")
fm3 = model.fit(smoothing_level=.5)
fcast_3 = fm3.forecast(9).rename(r"$\alpha=0.5$")

In [ ]:
fcast_2

In [ ]:
plt.figure(figsize=(10, 6))
(line0,) = plt.plot(train_data,  color="black")# plotting real temporal data
plt.plot(fm1.fittedvalues, color="blue")
(line1,) = plt.plot(fcast_1, marker=".", color="blue")
plt.plot(fm2.fittedvalues, color="red")
(line2,) = plt.plot(fcast_2, marker=".", color="red")
plt.plot(fm3.fittedvalues, color="green")
(line3,) = plt.plot(fcast_3, marker=".", color="green")
plt.legend([line1, line2, line3], [fcast_1.name, fcast_2.name, fcast_3.name])
plt.legend([line0, line1, line2, line3], ['true data',fcast_1.name, fcast_2.name, fcast_3.name])
# this will results in the forecasts with same value for all dates (9 out of the sample forecasts)
# because no trends are incorporated 

In [ ]:
# Holt exponential smoothing
# preparing the data; removing rows with zero values
df = pd.read_csv('Covid19_ts.csv',parse_dates=['Date'])
# remove zeros from the dataset, this will result in disonnection in the time series,
# thats why, we generate the time series manually..
df1 = df[df['Confirmed']> 0]
#get values without index
df1 = df1['Confirmed'].values
# convert from np array into list 
df1 = df1.tolist()
# select the first 215 values
df1 = df1[:215]
# generate time sequence
index = pd.date_range(start='01/31/2020', end='9/01/2020')
len(index)
# join datetime into the data
train_data = pd.Series(df1, index)

# linear smoothing
Holt_fit1 = Holt(train_data, initialization_method="estimated").fit(
    smoothing_level=0.9, smoothing_trend=0.2, optimized=False)
fcast_holt_1 = Holt_fit1.forecast(5).rename("Holt's linear trend")
#exponential smoothing 
Holt_fit2 = Holt(train_data, exponential=True, initialization_method="estimated").fit(
    smoothing_level=0.9, smoothing_trend=0.2, optimized=False)
fcast_holt_2 = Holt_fit2.forecast(5).rename("Exponential trend")

#Additive damped trend
Holt_fit3 = Holt(train_data, damped_trend=True, initialization_method="estimated").fit(
    smoothing_level=0.9, smoothing_trend=0.2)
fcast_holt_3 = Holt_fit3.forecast(5).rename("Additive damped trend")

plt.figure(figsize=(10, 6))
(line0,) = plt.plot(train_data,  color="black", label ='true data')
plt.plot(Holt_fit1.fittedvalues, color="blue", label ='linear smoothing')
(line1,) = plt.plot(fcast_holt_1, marker="o", color="blue")
#plt.plot(fcast_holt_2 , color="blue")
plt.plot(Holt_fit2.fittedvalues, color="red",label ='exponential smoothing')
(line2,) = plt.plot(fcast_holt_2, marker="o", color="red")
plt.plot(Holt_fit3.fittedvalues, color="green", label ='Additive damped trend')
(line3,) = plt.plot(fcast_holt_3, marker="o", color="green")
plt.legend(loc="upper left")
plt.show()
# forecasts show clearly the trend now..
